# <font color="#418FDE" size="6.5" uppercase>**Functional API**</font>

>Last update: 20260126.
    
By the end of this Lecture, you will be able to:
- Define Keras models using the Functional API with explicit Input and Output tensors. 
- Implement models with branching or skip connections using the Functional API graph structure. 
- Visualize and debug Functional models to ensure correct tensor shapes and connections. 


## **1. Functional Model Inputs Outputs**

### **1.1. Using Keras Input**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_01_01.jpg?v=1769435324" width="250">



>* Define a symbolic Input describing data shape
>* Explicit Input clarifies tensor flow and model expectations

>* Input objects describe each example’s structure
>* Clear shapes keep later layers compatible

>* Explicit inputs anchor and clarify computation flow
>* They ease shape reasoning, debugging, and collaboration



In [ ]:
#@title Python Code - Using Keras Input

# This script demonstrates Keras functional inputs.
# It focuses on using the keras Input object.
# Run cells sequentially to follow the explanation.

# TensorFlow is available by default in Google Colab.
# Uncomment the next line if running outside Colab.
# !pip install tensorflow==2.20.0.

# Import TensorFlow and Keras functional utilities.
import tensorflow as tf
from tensorflow.keras import layers, Model

# Set a deterministic random seed for reproducibility.
tf.random.set_seed(42)

# Print TensorFlow version in a single concise line.
print("TensorFlow version:", tf.__version__)

# Define the expected shape of one image example.
image_height, image_width, channels = 28, 28, 1

# Create an explicit Input tensor for grayscale images.
inputs = tf.keras.Input(shape=(image_height, image_width, channels))

# Confirm that the input tensor has the right shape.
print("Input tensor shape:", inputs.shape)

# Add a small Conv2D layer on top of the input.
x = layers.Conv2D(filters=8, kernel_size=(3, 3), activation="relu")(inputs)

# Downsample using MaxPooling to reduce spatial dimensions.
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Flatten the feature maps into a single vector.
x = layers.Flatten()(x)

# Add a Dense hidden layer for further processing.
x = layers.Dense(units=16, activation="relu")(x)

# Create the final output layer with ten units.
outputs = layers.Dense(units=10, activation="softmax")(x)

# Build the functional model from inputs and outputs.
model = Model(inputs=inputs, outputs=outputs, name="simple_functional_model")

# Display a short summary to inspect shapes.
model.summary(expand_nested=False, show_trainable=True)

# Create a small batch of dummy image data.
dummy_batch = tf.random.uniform(shape=(4, image_height, image_width, channels))

# Validate that the dummy batch shape matches the input.
assert dummy_batch.shape[1:] == inputs.shape[1:], "Shape mismatch detected."

# Run a forward pass to obtain model predictions.
predictions = model(dummy_batch, training=False)

# Print the shapes of inputs and outputs for clarity.
print("Dummy batch shape:", dummy_batch.shape)
print("Predictions shape:", predictions.shape)

# Show the first prediction vector to inspect output structure.
print("First prediction vector:", predictions[0].numpy())




### **1.2. Multiple Model Outputs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_01_02.jpg?v=1769435419" width="250">



>* Functional models can produce several outputs simultaneously
>* Shared layers feed separate heads for different predictions

>* One input can drive several related predictions
>* Shared layers boost efficiency, specialization, and generalization

>* Each output has its own targets, loss
>* Named outputs control training, weighting, and supervision



In [ ]:
#@title Python Code - Multiple Model Outputs

# This script shows Keras functional multiple outputs.
# It builds a tiny model with shared base layers.
# It trains briefly and prints shapes and predictions.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow and Keras modules.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Select device preference based on availability.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_type = "GPU"
else:
    device_type = "CPU"

# Print which device type will likely be used.
print("Using device type:", device_type)

# Define input shape for simple numeric features.
input_dim = 8
num_samples = 64
num_classes = 3

# Create random input data with correct shape.
X = np.random.rand(num_samples, input_dim).astype("float32")

# Create classification labels as integers.
class_labels = np.random.randint(num_classes, size=num_samples)

# Create regression targets as continuous values.
reg_targets = np.random.rand(num_samples, 1).astype("float32")

# One hot encode classification labels.
class_targets = keras.utils.to_categorical(class_labels, num_classes)

# Validate shapes before building the model.
print("Input shape:", X.shape)
print("Class target shape:", class_targets.shape)
print("Reg target shape:", reg_targets.shape)

# Define the functional model input tensor.
inputs = keras.Input(shape=(input_dim,), name="features")

# Build a small shared dense representation.
shared = layers.Dense(16, activation="relu", name="shared_dense")(inputs)

# Add a second shared layer for richer features.
shared = layers.Dense(8, activation="relu", name="shared_dense_two")(shared)

# Define classification output head from shared features.
class_output = layers.Dense(
    num_classes,
    activation="softmax",
    name="class_output",
)(shared)

# Define regression output head from shared features.
reg_output = layers.Dense(
    1,
    activation="linear",
    name="reg_output",
)(shared)

# Create the functional model with two outputs.
model = keras.Model(
    inputs=inputs,
    outputs=[class_output, reg_output],
    name="multi_output_model",
)

# Print a short summary using minimal lines.
model.summary(print_fn=lambda x: None)
print("Model outputs:", [t.name for t in model.outputs])

# Compile model with separate losses and metrics.
model.compile(
    optimizer="adam",
    loss={"class_output": "categorical_crossentropy", "reg_output": "mse"},
    metrics={"class_output": "accuracy", "reg_output": "mae"},
)

# Train briefly with small epochs and silent logs.
history = model.fit(
    X,
    {"class_output": class_targets, "reg_output": reg_targets},
    epochs=3,
    batch_size=16,
    verbose=0,
)

# Evaluate model once to check both outputs.
results = model.evaluate(
    X,
    {"class_output": class_targets, "reg_output": reg_targets},
    verbose=0,
)

# Print evaluation results in a compact way.
print("Evaluation results:", results)

# Run prediction on a small batch of inputs.
sample_inputs = X[:2]
class_pred, reg_pred = model.predict(sample_inputs, verbose=0)

# Print shapes of the two prediction outputs.
print("Pred class shape:", class_pred.shape)
print("Pred reg shape:", reg_pred.shape)

# Print first sample predictions for both heads.
print("First sample class probs:", class_pred[0])
print("First sample reg value:", reg_pred[0])




### **1.3. Constructing Keras Models**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_01_03.jpg?v=1769435458" width="250">



>* Imagine data flowing through layers as transformations
>* Connect layers into a graph from input to output

>* Map inputs through layers to final prediction
>* Explicitly pair input and output tensors as model

>* Same input-output pattern scales to any architecture
>* Explicit mappings make models clearer, shareable, debuggable



In [ ]:
#@title Python Code - Constructing Keras Models

# This script shows Keras Functional API basics.
# It focuses on explicit inputs and outputs.
# Run cells sequentially inside Google Colab.

# TensorFlow is available in this environment already.
# Uncomment next line if running outside this notebook.
# !pip install tensorflow==2.20.0.

# Import TensorFlow and Keras modules.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
tf.random.set_seed(7)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define input shape for simple numeric features.
num_features = 4

# Create an explicit Input tensor for the model.
inputs = keras.Input(shape=(num_features,), name="house_features")

# Add first Dense layer transforming the input tensor.
x = layers.Dense(8, activation="relu", name="dense_1")(inputs)

# Add second Dense layer for further feature mixing.
x = layers.Dense(4, activation="relu", name="dense_2")(x)

# Create final prediction layer producing one output value.
outputs = layers.Dense(1, activation="linear", name="price_output")(x)

# Build the Functional model from inputs and outputs.
model = keras.Model(inputs=inputs, outputs=outputs, name="price_model")

# Show a short model summary for debugging shapes.
model.summary(expand_nested=False, show_trainable=True)

# Create a tiny dummy dataset with correct feature shape.
import numpy as np
num_samples = 6

# Generate small deterministic feature matrix.
features = np.arange(num_samples * num_features, dtype="float32")
features = features.reshape((num_samples, num_features))

# Generate simple target values as small float numbers.
targets = np.linspace(100000.0, 160000.0, num_samples, dtype="float32")

# Validate that feature and target shapes are compatible.
assert features.shape == (num_samples, num_features)
assert targets.shape == (num_samples,)

# Compile the model with mean squared error loss.
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train briefly with silent verbose setting to avoid logs.
history = model.fit(features, targets, epochs=5, batch_size=2, verbose=0)

# Use the model to predict prices for the same features.
predictions = model.predict(features, verbose=0)

# Print shapes to confirm input and output tensor dimensions.
print("Input shape:", features.shape)
print("Output shape:", predictions.shape)

# Print a few example predictions alongside true targets.
for i in range(min(3, num_samples)):
    print("Sample", i, "target:", float(targets[i]), "pred:", float(predictions[i, 0]))



## **2. Functional Graph Connections**

### **2.1. Calling Layers in Functional API**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_02_01.jpg?v=1769435498" width="250">



>* Layers act as functions transforming input tensors
>* Graph view enables branches, merges, skip connections

>* Layer calls show changing tensor shapes clearly
>* Graph wiring supports complex, real-world workflows

>* Layers on tensors enable flexible graph patterns
>* Easily build shared, branched, and merged pathways



In [ ]:
#@title Python Code - Calling Layers in Functional API

# This script shows Keras Functional API basics.
# It focuses on calling layers like tensor functions.
# We also build a small branching and skip model.

# Install TensorFlow if needed in other environments.
# pip install tensorflow==2.20.0.

# Import TensorFlow and Keras modules.
import tensorflow as tf

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Set deterministic seeds for reproducibility.
tf.random.set_seed(7)

# Define input tensor for simple numeric features.
inputs = tf.keras.Input(shape=(4,), name="numeric_input")

# Call a Dense layer on the input tensor.
base = tf.keras.layers.Dense(
    8, activation="relu", name="base_dense"
)(inputs)

# Branch one processes base features for task A.
branch_a = tf.keras.layers.Dense(
    4, activation="relu", name="branch_a_dense"
)(base)

# Branch two processes base features for task B.
branch_b = tf.keras.layers.Dense(
    4, activation="relu", name="branch_b_dense"
)(base)

# Create a skip connection from input to merge point.
skip = tf.keras.layers.Dense(
    4, activation="linear", name="skip_from_input"
)(inputs)

# Concatenate both branches and the skip tensor.
merged = tf.keras.layers.Concatenate(name="merge_branches")(
    [branch_a, branch_b, skip]
)

# Add a shared layer called on merged tensor.
shared = tf.keras.layers.Dense(
    6, activation="relu", name="shared_dense"
)(merged)

# Output for main prediction task.
output_main = tf.keras.layers.Dense(
    1, activation="sigmoid", name="main_output"
)(shared)

# Output for auxiliary prediction task.
output_aux = tf.keras.layers.Dense(
    1, activation="sigmoid", name="aux_output"
)(branch_b)

# Build the Functional model from inputs and outputs.
model = tf.keras.Model(
    inputs=inputs, outputs=[output_main, output_aux], name="branch_skip_model"
)

# Show a short summary to inspect graph shapes.
model.summary(line_length=80, expand_nested=False)

# Create a tiny batch of dummy numeric data.
x_sample = tf.ones(shape=(3, 4), dtype=tf.float32)

# Validate input shape before calling the model.
assert x_sample.shape[1] == 4

# Call the model on the sample to get predictions.
y_main_pred, y_aux_pred = model(x_sample, training=False)

# Print shapes to see how tensors flowed.
print("Input shape:", x_sample.shape)

# Print main output tensor shape.
print("Main output shape:", y_main_pred.shape)

# Print auxiliary output tensor shape.
print("Aux output shape:", y_aux_pred.shape)



### **2.2. Merging Add and Concatenate**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_02_02.jpg?v=1769435534" width="250">



>* Addition blends same-shaped branches elementwise into one
>* Concatenation stacks features, expanding dimensions and capacity

>* Addition sums same-shaped feature maps, enforcing consensus
>* Best for residual refinements; preserves shape, blends sources

>* Concatenation keeps branch features separate for later mixing
>* It enlarges representations, increasing parameters and computation



In [ ]:
#@title Python Code - Merging Add and Concatenate

# This script shows Keras functional merging operations.
# We compare Add and Concatenate on simple branches.
# Focus on shapes and connections not training performance.

# Install TensorFlow if missing in some environments.
# !pip install tensorflow==2.20.0.

# Import TensorFlow and Keras functional utilities.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
tf.random.set_seed(7)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a small input tensor for dense features.
inputs = keras.Input(shape=(4,), name="numeric_input")

# Build first branch with one Dense layer.
branch_one = layers.Dense(3, activation="relu", name="b1_dense")(
    inputs
)

# Build second branch with different Dense layer.
branch_two = layers.Dense(3, activation="relu", name="b2_dense")(
    inputs
)

# Confirm both branch output shapes are identical.
print("Branch one shape:", branch_one.shape)
print("Branch two shape:", branch_two.shape)

# Merge branches using elementwise Add layer.
added = layers.Add(name="merge_add")([
    branch_one,
    branch_two,
])

# Merge branches using feature Concatenate layer.
concatenated = layers.Concatenate(name="merge_concat")([
    branch_one,
    branch_two,
])

# Show shapes after Add and Concatenate merges.
print("Added merge shape:", added.shape)
print("Concatenated merge shape:", concatenated.shape)

# Build model that outputs both merged tensors.
model = keras.Model(
    inputs=inputs,
    outputs=[added, concatenated],
    name="add_vs_concat_model",
)

# Display a concise model summary for debugging.
model.summary(line_length=80, expand_nested=False)

# Create a tiny batch of example numeric data.
example_batch = tf.constant(
    [[1.0, 2.0, 3.0, 4.0], [0.5, 0.5, 0.5, 0.5]],
    dtype=tf.float32,
)

# Run the model once to get both outputs.
added_out, concat_out = model(example_batch)

# Print small slices to compare values and shapes.
print("Added output sample:", added_out[0].numpy())
print("Concat output sample:", concat_out[0].numpy())

# Final print to emphasize dimensionality difference.
print("Final shapes -> add:", added_out.shape, "concat:", concat_out.shape)



### **2.3. Designing Skip Connections**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_02_03.jpg?v=1769435574" width="250">



>* Skip connections bypass layers to preserve information, gradients
>* Functional API wires earlier tensors directly into later operations

>* Plan what to skip and how to merge
>* Match tensor shapes and wire skips explicitly

>* Skip connections enable multi-scale models across domains
>* Functional API makes experimenting with skip patterns easy



In [ ]:
#@title Python Code - Designing Skip Connections

# This script shows Keras functional skip connections.
# It builds and inspects a simple residual style model.
# Focus on shapes and graph style tensor wiring.

# !pip install tensorflow==2.20.0.

# Import required modules from TensorFlow.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
import numpy as np
import random as py_random
np.random.seed(7)
py_random.seed(7)
tf.random.set_seed(7)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a small input shape for demonstration.
input_shape = (28, 28, 1)

# Create an Input tensor for the functional model.
inputs = keras.Input(shape=input_shape, name="image_input")

# Apply a first convolution block on the input.
x = layers.Conv2D(16, (3, 3), padding="same", activation="relu")(inputs)

# Store this tensor as the skip connection source.
skip_tensor = x

# Add a second convolution block for deeper features.
x = layers.Conv2D(16, (3, 3), padding="same", activation="relu")(x)

# Ensure shapes match before adding the skip connection.
if skip_tensor.shape != x.shape:
    raise ValueError("Skip and main tensors must share shape.")

# Merge skip and main path using elementwise addition.
merged = layers.Add(name="skip_add")([skip_tensor, x])

# Continue with pooling and flattening after merge.
x = layers.MaxPooling2D(pool_size=(2, 2))(merged)

# Flatten spatial dimensions into a feature vector.
x = layers.Flatten()(x)

# Add a small dense layer for classification.
outputs = layers.Dense(10, activation="softmax", name="class_output")(x)

# Build the functional model from inputs and outputs.
model = keras.Model(inputs=inputs, outputs=outputs, name="skip_demo_model")

# Display a concise model summary for shape inspection.
model.summary(line_length=80, expand_nested=False)

# Create a tiny batch of random images for testing.
sample_batch = np.random.rand(4, 28, 28, 1).astype("float32")

# Run a forward pass to verify output shape.
preds = model(sample_batch, training=False)

# Print key tensor shapes to highlight skip behavior.
print("Input batch shape:", sample_batch.shape)
print("Skip tensor shape:", skip_tensor.shape)
print("Merged tensor shape:", merged.shape)
print("Output batch shape:", preds.shape)



## **3. Inspecting Functional Models**

### **3.1. Reading Model Summaries**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_03_01.jpg?v=1769435609" width="250">



>* Model summary shows data flow through layers
>* Trace branches and merges to verify architecture

>* Track how tensor shapes change between layers
>* Use shape patterns to spot errors quickly

>* Use summaries to track complex layer connections
>* Check merges and skips to avoid subtle bugs



In [ ]:
#@title Python Code - Reading Model Summaries

# This script shows how to read model summaries.
# It uses a small Functional API example model.
# Focus on shapes and connections in the summary.

# !pip install tensorflow==2.20.0.

# Import TensorFlow and Keras modules.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set deterministic seeds for reproducibility.
tf.random.set_seed(7)

# Define an image input for a tiny branch.
image_input = keras.Input(shape=(8, 8, 1), name="image_input")

# Define a tabular input for a second branch.
tabular_input = keras.Input(shape=(4,), name="tabular_input")

# Build a small convolutional block for images.
x = layers.Conv2D(4, (3, 3), activation="relu")(image_input)

# Flatten image features to a vector.
x = layers.Flatten(name="image_flatten")(x)

# Build a dense block for tabular data.
y = layers.Dense(8, activation="relu", name="tabular_dense")(tabular_input)

# Add a skip connection on the tabular branch.
skip = layers.Dense(8, activation="relu", name="tabular_skip")(tabular_input)

# Concatenate main and skip tabular features.
y = layers.Concatenate(name="tabular_concat")([y, skip])

# Merge image and tabular branches together.
merged = layers.Concatenate(name="fusion_concat")([x, y])

# Add a small hidden layer after fusion.
hidden = layers.Dense(16, activation="relu", name="fusion_dense")(merged)

# Define a final classification output layer.
output = layers.Dense(3, activation="softmax", name="class_output")(hidden)

# Create the Functional model from inputs and output.
model = keras.Model(inputs=[image_input, tabular_input], outputs=output)

# Print a compact model summary for inspection.
model.summary(line_length=80, expand_nested=False)

# Show the model inputs and outputs shapes clearly.
print("Inputs:", [inp.shape for inp in model.inputs])
print("Output:", model.output.shape)



### **3.2. Visualizing Model Graphs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_03_02.jpg?v=1769435638" width="250">



>* Graph diagrams show layers and tensor flow clearly
>* They reveal branches, merges, and overall architecture

>* Graphs clarify branching, merging, and skip connections
>* They help catch missing, misaligned, or wrong branches

>* Graphs show tensor shapes and layer compatibility
>* They pinpoint shape errors and guide architecture fixes



In [ ]:
#@title Python Code - Visualizing Model Graphs

# This script shows Keras functional model visualization.
# It focuses on graph structure and tensor shapes.
# Run cells in order to follow the explanation.

# Optional install for specific TensorFlow version in Colab.
# !pip install -q tensorflow==2.20.0.

# Import TensorFlow and Keras utilities.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# Set random seeds for deterministic behavior.
tf.random.set_seed(7)

# Define input for image like data.
image_input = keras.Input(shape=(28, 28, 1), name="image_input")

# Define a small convolution block.
x = layers.Conv2D(8, (3, 3), activation="relu")(image_input)

# Add max pooling to reduce spatial size.
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Add another convolution layer for depth.
x = layers.Conv2D(16, (3, 3), activation="relu")(x)

# Flatten features for dense layers.
flat_features = layers.Flatten(name="flat_features")(x)

# Define a second input branch for numeric data.
meta_input = keras.Input(shape=(4,), name="meta_input")

# Process metadata with a dense layer.
meta_features = layers.Dense(8, activation="relu")(meta_input)

# Concatenate image and metadata features.
merged = layers.Concatenate(name="merge_features")(
    [flat_features, meta_features]
)

# Add a dense layer after merging.
hidden = layers.Dense(16, activation="relu")(merged)

# Define output for binary classification.
output = layers.Dense(1, activation="sigmoid", name="output")(hidden)

# Build the functional model from inputs and outputs.
model = keras.Model(inputs=[image_input, meta_input], outputs=output)

# Print a short summary to inspect shapes.
model.summary(line_length=80, expand_nested=False)

# Visualize model graph to a PNG file.
keras.utils.plot_model(
    model,
    to_file="functional_model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
)

# Confirm that the plot file was created successfully.
import os
print("Plot file exists:", os.path.exists("functional_model.png"))




### **3.3. Debugging shape mismatches**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_03/Lecture_B/image_03_03.jpg?v=1769435694" width="250">



>* Shape mismatches happen when tensor dimensions disagree
>* Trace one example’s shape through layers to debug

>* Compare expected and actual tensor shapes around errors
>* Align shapes with pooling, projections, or reshaping

>* Describe each tensor’s meaning and expected shape
>* Use tiny test inputs to locate mismatches



In [ ]:
#@title Python Code - Debugging shape mismatches

# This script shows debugging shape mismatches.
# It uses Keras Functional API with tensors.
# Focus on inspecting shapes and fixing errors.

# !pip install tensorflow==2.20.0.

# Import TensorFlow and Keras modules.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic random seeds.
import numpy as np
np.random.seed(7)
tf.random.set_seed(7)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Define an image input tensor.
image_input = keras.Input(shape=(28, 28, 1), name="image_input")

# Apply a small convolution block.
x = layers.Conv2D(8, (3, 3), activation="relu", padding="same")(image_input)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Intentionally create a mismatched skip tensor.
skip = layers.Conv2D(8, (1, 1), activation="relu")(image_input)

# Show shapes before the faulty addition.
print("Main branch shape:", x.shape)
print("Skip branch shape:", skip.shape)

# Demonstrate what a bad Add would receive.
print("Attempting Add would need equal shapes.")

# Fix mismatch by downsampling the skip path.
fixed_skip = layers.MaxPooling2D(pool_size=(2, 2))(skip)

# Confirm shapes now match for addition.
print("Fixed skip shape:", fixed_skip.shape)

# Safely add tensors with matching shapes.
added = layers.Add()([x, fixed_skip])

# Flatten and add a small Dense head.
flat = layers.Flatten()(added)
output = layers.Dense(10, activation="softmax", name="class_output")(flat)

# Build the Functional model object.
model = keras.Model(inputs=image_input, outputs=output, name="debug_model")

# Print a concise summary to inspect shapes.
model.summary(expand_nested=False, show_trainable=True)

# Create tiny synthetic image data.
sample_images = np.random.rand(4, 28, 28, 1).astype("float32")

# Run a forward pass to verify everything.
preds = model(sample_images, training=False)

# Print final prediction shape for confirmation.
print("Prediction batch shape:", preds.shape)



# <font color="#418FDE" size="6.5" uppercase>**Functional API**</font>


In this lecture, you learned to:
- Define Keras models using the Functional API with explicit Input and Output tensors. 
- Implement models with branching or skip connections using the Functional API graph structure. 
- Visualize and debug Functional models to ensure correct tensor shapes and connections. 

In the next Module (Module 4), we will go over 'Training Workflows'